# Calibration with initial model memory states as parameters


## About this document


In [ ]:
from swift2.doc_helper import pkg_versions_info

print(pkg_versions_info("This document was generated from a jupyter notebook"))


This vignette will illustrate how to define two _meta-parameters_, _S0_ and _R0_, controlling the initial level of stores in the GR4J model, as fraction of the store capacities.

We'll load a simple catchment with one subarea only; the feature applies equally to catchment with multiple sub-areas

## Imports

In [ ]:
import numpy as np
import pandas as pd
import swift2.doc_helper as std
import swift2.parameteriser as sp
import swift2.system as ssy

In [ ]:
from cinterop.timeseries import xr_ts_end, xr_ts_start
from swift2.classes import CompositeParameteriser

In [ ]:
from swift2.utils import as_xarray_series, mk_full_data_id
from swift2.vis import plot_two_series
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

## Model building

We will use sample daily data for the Ovens Catchment at Bright (VIC), sourced from [Victoria's Water Measurement Information System](https://data.water.vic.gov.au/WMIS) and [SILO patch point data](https://www.longpaddock.qld.gov.au/silo/point-data/). While very convenient for a tutorial, remember you should source your own data for your modelling needs.

In [ ]:
model_id = "GR4J"
simul_start = pd.Timestamp(
    "2016-10-01"
)  # we pick a time where usually catchments are wetter in VIC and there is higher streamflow typically
simul_end = pd.Timestamp("2018-12-31")
ms = std.create_subarea_simulation(
    data_id="Ovens-Bright",
    simul_start=simul_start,
    simul_end=simul_end,
    model_id=model_id,
    tstep="daily",
    varname_rain="P",
    varname_pet="E",
    data_rain_id="rain",
    data_evap_id="pet",
)

We devise a few model states of interest to illustrate the role of initial conditions, and we set some them to be recorded to time series in upcoming model simulations

In [ ]:
gr4jModelVars = ssy.runoff_model_var_ids(model_id)
print(gr4jModelVars)

In [ ]:
def mk_varid(short_id):
    element_id = "subarea.Subarea"
    return mk_full_data_id(element_id, short_id)


runoff_id = mk_varid("runoff")
s_store_id = mk_varid("S")
r_store_id = mk_varid("R")
ms.record_state([runoff_id, s_store_id, r_store_id])

We set up a relatively short runtime span of barely two years, with a significant starting observed streamflow as we'll see.

In [ ]:
obs_streamflow_mld = std.sample_series("Ovens-Bright", "streamflow")
obs_streamflow_mld[obs_streamflow_mld < -1] = np.nan
area_km2 = 495
obs_runoff = obs_streamflow_mld / area_km2  # ML/day to mm/

In [ ]:
s = simul_start
w = s # i.e. no warmup! too short a simulation span to afford a warmup
# e = s + pd.DateOffset(days=90)
e = simul_end
obs_runoff = obs_runoff[slice(s, e)]
obs_runoff.plot()
plt.title("Observed runoff depth at Bright, VIC")
plt.ylabel("runoff, mm/day");

In [ ]:
ms.set_simulation_span(s, e)

Looking at the observed streamflow time series above, we definitely have a wet start to the period of interest above. Let's apply some default model parameters to the model, and see the default output runoff.

In [ ]:
pspec_gr4j = std.get_free_params(model_id)
# replace the short param names x1 etc. with subarea identifiers
pspec_gr4j["Name"] = mk_varid(pspec_gr4j["Name"])
p = sp.create_parameteriser("Generic", pspec_gr4j)
p

We are using the daily time step, so for `GR4J` we need to adjust two things to make some settings consistent

In [ ]:
from swift2.doc_helper import configure_daily_gr4j
configure_daily_gr4j(ms)  # probably the default anyway, but safety first

In [ ]:
p.set_max_parameter_value("subarea.Subarea.x4", 10)
p.set_parameter_value("subarea.Subarea.x4", 3)
p

In [ ]:
p.apply_sys_config(ms)

We get a time series of the store _S_ if we run it at this point; the starting value is zero.

In [ ]:

ms.exec_simulation()
tts = ms.get_recorded(s_store_id)
tts = tts.squeeze(drop=True)
g = tts.plot.line(add_legend=True, figsize=(10, 5))
plt.title("GR4J S store. No state initializer");

In [ ]:
calc_runoff = ms.get_recorded(runoff_id).squeeze(drop=True)

We will show the same type of plot a few times, so let'd define a shorthand function:

In [ ]:
def ts_runoff_mod_calc(obs_runoff, calc_runoff):
    plot_two_series(
        obs_runoff,
        calc_runoff,
        start_time=xr_ts_start(calc_runoff),
        end_time=xr_ts_end(calc_runoff),
        names=["observed", "modelled"],
        xlab="time",
        ylab="runoff (mm)",
        figsize=(10, 5),
    )


ts_runoff_mod_calc(obs_runoff, calc_runoff)

Let's define _S0_ and _R0_ parameters such that for each GR4J model instance (only one in this case), _S = S0 * x1_ and _R = R0 * x3_

In [ ]:
p_states = sp.linear_parameteriser(
    ["S0", "R0"],
    ["S", "R"],
    ["x1", "x3"],
    [0.04, 0.04],
    [0.98, 0.98],
    [0.4, 0.4],
    "each subarea",
)

If one applies this parameteriser _p_states_ to the system, the the _S_ store is set to the expected value relative to the model state _x1_.

In [ ]:
ms.get_state_value(s_store_id)

We first **clone** the model simulation, to keep a clean baseline `ms`. This may seem superfluous, but we encourage this practice in exploratory model analysis to avoid unwanted "dregs" of model configurations. In particular it is easy to not realise (or forget) that a model simulation has a state initialiser set.

In [ ]:
ms_tmp = ms.clone()

In [ ]:
p_states.apply_sys_config(ms_tmp)
ms_tmp.get_state_value([s_store_id, mk_varid("x1")])

**However** this is a one off operation. If executing the simulation, the time series of _S_ still starts at zero, because the resetting the model overrides the state _S_:

In [ ]:
ms_tmp.exec_simulation()
tts = ms_tmp.get_recorded(s_store_id)
tts = tts.squeeze(drop=True)
g = tts.plot.line(add_legend=True, figsize=(10, 5))
plt.title("GR4J S store; incomplete store initialization");

To be able to calibrate initial states as parameters, _p_states_ needs to be wrapped into a parameteriser that overrides the "reset" step of any model simulation that happens (by default anyway) ad every model simulation.

### Creating a state initialisation parameteriser

You need to define a new parameteriser, that makes sure that the model is reset to the expected initial value. This is done by calling the method **make_state_init_parameteriser**.

In [ ]:
init_parameteriser = p_states.make_state_init_parameteriser()
# clone again, new clean slate...
ms_statinitparams = ms.clone()
init_parameteriser.apply_sys_config(ms_statinitparams)

If this parameteriser is applied to the simulation (as would be by an optimiser) then the subsequent model simulation will be with an initialised model state.

In [ ]:
ms_statinitparams.exec_simulation()
tts = ms_statinitparams.get_recorded(s_store_id)
tts = tts.squeeze(drop=True)
g = tts.plot.line(add_legend=True, figsize=(10, 5))
plt.title("GR4J S store, with a proper state initializer");

In [ ]:
calc_runoff = ms_statinitparams.get_recorded(runoff_id).squeeze(drop=True)
ts_runoff_mod_calc(obs_runoff, calc_runoff)

We do notice that the calculated runoff gets off "zero" earlier than before, but still not reflecting well the observations. Let's create a calibration such that these initial states become parameters that can be fitted.

There is logic in keeping the two previous steps in defining a parameteriser as separate, hence this present vignette emphasizes the importance of these two steps.

## Calibration

Once you have defined this state initialisation parameteriser using `make_state_init_parameteriser` above, you can define a calibration objective the usual way. This vignette does not get into a detailed explanation of setting up a calibration; please refer to other vignettes (getting started).

Calibrating initial states alone may make some sense in some contexts, but in this case we create a parameteriser with 6 parameters, aggregate of the two state initialisers and 4 usual parameter for GR4J:

In [ ]:
p_composite = CompositeParameteriser.concatenate(p, init_parameteriser)
print(p_composite)

We get back to using our baseline simulation `ms` to build a calibration objective.

In [ ]:
ms.describe()

In [ ]:
objective = ms.create_objective(runoff_id, obs_runoff, "NSE", w, e)

In [ ]:
objective.get_score(p_composite)

In [ ]:
calc_runoff = ms.get_recorded(runoff_id).squeeze(drop=True)
ts_runoff_mod_calc(obs_runoff, calc_runoff)

### Calibration without initial states as parameters

We have defined the composite parameteriser with 6 parameters. To illustrate the difference in model performance and goodness of fit, let us first do a vanilla calibration with only the usual 4 parameters. 

In [ ]:
optim_noinit = objective.create_sce_optim_swift(population_initialiser=p)
optim_noinit.set_calibration_logger('')

In [ ]:
%%time
results = optim_noinit.execute_optimisation()

We define a shorthand function to plot the calibration process, which we will do twice:

In [ ]:
def plot_calib_evolution(optimiser):
    opt_log = optimiser.extract_optimisation_log(fitness_name = "NSE")
    geom_ops = opt_log.subset_by_message(pattern= 'Initial.*|Reflec.*|Contrac.*|Add.*') # same as default argument, but to be explicit
    p_var_ids = ['x1','x2','x3','x4']
    remap = {f'subarea.Subarea.{name}': name for name in p_var_ids}
    geom_ops.rename_columns(remap)
    from swift2.vis import OptimisationPlots
    
    v = OptimisationPlots(geom_ops)
    g = v.parameter_evolution(p_var_ids[0], obj_lims=[0,1])
    plt.gcf().set_size_inches(10,8);

plot_calib_evolution(optim_noinit)

In [ ]:
s = results.get_best_score("NSE")
s

In [ ]:
ms_tmp = ms.clone() # keep the baseline clean
ms_tmp.remove_state_initialisers()
s.apply_sys_config(ms_tmp)
ms_tmp.exec_simulation()

We notice above that the store capacity model parameter _x1_ is low; the optimiser is trying to "lift" the calculated runoff time series by lowering the first store capacity, illustrating the impact of initial conditions when calibrating on short periods

In [ ]:
calc_runoff = ms_tmp.get_recorded(runoff_id).squeeze(drop=True)
ts_runoff_mod_calc(obs_runoff, calc_runoff)

0.20 NSE seems a bit low, visually, but if we cross check independendly indeed this mismatch in initial condition lowers the score that much. 

In [ ]:
import numpy as np
import pandas as pd


def nash_sutcliffe_efficiency(observed: pd.Series, modelled: pd.Series) -> float:
    """
    Calculates the Nash-Sutcliffe Efficiency (NSE) between two pandas Series.

    Args:
        observed (pd.Series): The observed values.
        modelled (pd.Series): The modelled values.

    Returns:
        float: The Nash-Sutcliffe Efficiency.
    """
    if len(observed) != len(modelled):
        raise ValueError("Observed and modelled series must have the same length.")

    observed_mean = observed.mean()
    numerator = np.sum((observed - modelled)**2)
    denominator = np.sum((observed - observed_mean)**2)

    if denominator == 0:
        return np.nan  # or np.inf if you want to return infinity

    nse = 1 - (numerator / denominator)
    return nse

In [ ]:
nash_sutcliffe_efficiency(obs_runoff, calc_runoff)

### Calibration with initial states as parameters

In [ ]:
score = objective.get_score(p_composite)
score

In [ ]:
optim_with_init = objective.create_sce_optim_swift(population_initialiser=p_composite)

In [ ]:
%%time
results = optim_with_init.execute_optimisation()

In [ ]:
results.get_best_score("NSE", convert_to_py=True)

In [ ]:
s = results.get_best_score("NSE")

In [ ]:
ms_allparams = ms.clone()
s.apply_sys_config(ms_allparams)
ms_allparams.exec_simulation()

In [ ]:
calc_runoff = ms_allparams.get_recorded(runoff_id).squeeze(drop=True)

ts_runoff_mod_calc(obs_runoff, calc_runoff)

In [ ]:
nash_sutcliffe_efficiency(obs_runoff, calc_runoff)